# Model Register and Endpoint



## Startup cells

In [0]:
# Set environment variables for sagemaker_studio imports

import os
os.environ['DataZoneProjectId'] = 'd93gwzwoenvv6x'
os.environ['DataZoneDomainId'] = 'dzd-5ah6gnp909njop'
os.environ['DataZoneEnvironmentId'] = '6hjtoxqai3bb8p'
os.environ['DataZoneDomainRegion'] = 'us-east-2'

# create both a function and variable for metadata access
_resource_metadata = None

def _get_resource_metadata():
    global _resource_metadata
    if _resource_metadata is None:
        _resource_metadata = {
            "AdditionalMetadata": {
                "DataZoneProjectId": "d93gwzwoenvv6x",
                "DataZoneDomainId": "dzd-5ah6gnp909njop",
                "DataZoneEnvironmentId": "6hjtoxqai3bb8p",
                "DataZoneDomainRegion": "us-east-2",
            }
        }
    return _resource_metadata
metadata = _get_resource_metadata()

In [0]:
"""
Logging Configuration

Purpose:
--------
This sets up the logging framework for code executed in the user namespace.
"""

from typing import Optional


def _set_logging(log_dir: str, log_file: str, log_name: Optional[str] = None):
    import os
    import logging
    from logging.handlers import RotatingFileHandler

    level = logging.INFO
    max_bytes = 5 * 1024 * 1024
    backup_count = 5

    # fallback to /tmp dir on access, helpful for local dev setup
    try:
        os.makedirs(log_dir, exist_ok=True)
    except Exception:
        log_dir = "/tmp/kernels/"

    os.makedirs(log_dir, exist_ok=True)
    log_path = os.path.join(log_dir, log_file)

    logger = logging.getLogger() if not log_name else logging.getLogger(log_name)
    logger.handlers = []
    logger.setLevel(level)

    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")

    # Rotating file handler
    fh = RotatingFileHandler(filename=log_path, maxBytes=max_bytes, backupCount=backup_count, encoding="utf-8")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    logger.info(f"Logging initialized for {log_name}.")


_set_logging("/var/log/computeEnvironments/kernel/", "kernel.log")
_set_logging("/var/log/studio/data-notebook-kernel-server/", "metrics.log", "metrics")

In [0]:
import logging
from sagemaker_studio import ClientConfig, sqlutils, sparkutils, dataframeutils

logger = logging.getLogger(__name__)
logger.info("Initializing sparkutils")
spark = sparkutils.init()
logger.info("Finished initializing sparkutils")

In [0]:
def _reset_os_path():
    """
    Reset the process's working directory to handle mount timing issues.
    
    This function resolves a race condition where the Python process starts
    before the filesystem mount is complete, causing the process to reference
    old mount paths and inodes. By explicitly changing to the mounted directory
    (/home/sagemaker-user), we ensure the process uses the correct, up-to-date
    mount point.
    
    The function logs stat information (device ID and inode) before and after
    the directory change to verify that the working directory is properly
    updated to reference the new mount.
    
    Note:
        This is executed at module import time to ensure the fix is applied
        as early as possible in the kernel initialization process.
    """
    try:
        import os
        import logging

        logger = logging.getLogger(__name__)
        logger.info("---------Before------")
        logger.info("CWD: %s", os.getcwd())
        logger.info("stat('.'): %s %s", os.stat('.').st_dev, os.stat('.').st_ino)
        logger.info("stat('/home/sagemaker-user'): %s %s", os.stat('/home/sagemaker-user').st_dev, os.stat('/home/sagemaker-user').st_ino)

        os.chdir("/home/sagemaker-user")

        logger.info("---------After------")
        logger.info("CWD: %s", os.getcwd())
        logger.info("stat('.'): %s %s", os.stat('.').st_dev, os.stat('.').st_ino)
        logger.info("stat('/home/sagemaker-user'): %s %s", os.stat('/home/sagemaker-user').st_dev, os.stat('/home/sagemaker-user').st_ino)
    except Exception as e:
        logger.exception(f"Failed to reset working directory: {e}")

_reset_os_path()

## Notebook

In [0]:
#---------------------------------------Creacion Grupo de Modelo--------------------------------------
import boto3
import time
from sagemaker import get_execution_role, image_uris
from botocore.exceptions import ClientError # Importación clave para manejo de errores

# -------------Informacion-------------------
region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client('sagemaker', region_name=region)

# Definición de variables
model_package_group_name = "Fleet-Model-Maintenance"

# -----------------Registro del grupo de modelos (Robusto)------------
model_package_group_input_dict = {
"ModelPackageGroupName" : model_package_group_name,
"ModelPackageGroupDescription" : "Group of model for maintenance prediction"
}

try:
    create_model_package_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
    print('ModelPackageGroup Creado. ARN: {}'.format(create_model_package_group_response['ModelPackageGroupArn']))
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceInUse':
        print(f"ModelPackageGroup ya existe: {model_package_group_name}")
    else:
        raise e



sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


ModelPackageGroup Creado. ARN: arn:aws:sagemaker:us-east-2:491934530980:model-package-group/Fleet-Model-Maintenance

 Registrando nueva versión en el grupo Fleet-Model-Maintenance...


ModelPackage Version ARN : arn:aws:sagemaker:us-east-2:491934530980:model-package/Fleet-Model-Maintenance/1
Estado inicial: Completed

 El registro de la versión del modelo ha finalizado con éxito.
ARN del Modelo Registrado: arn:aws:sagemaker:us-east-2:491934530980:model-package/Fleet-Model-Maintenance/1


In [0]:
#-----------------------------------Registro de versión------------------------------------------
import boto3
import time
from sagemaker import get_execution_role, image_uris
from botocore.exceptions import ClientError # Importación clave para manejo de errores

#Definiciones
model_package_group_name = "Fleet-Model-Maintenance"
model_url = 's3://fleet-sagemaker-maintenance/Models/model.tar.gz'
xgboost_version = '1.7-1'
py_version = 'py3'

image_uri = image_uris.retrieve(
    framework='xgboost',
    region=region,
    version=xgboost_version,
    py_version=py_version
    # Eliminamos 'instance_type' de retrieve
)

modelpackage_inference_specification =  {
    "InferenceSpecification": {
        "Containers": [
            {
                "Image": image_uri,
                "ModelDataUrl": model_url,
            }
        ],
        "SupportedContentTypes": ["text/csv"], 
        "SupportedResponseMIMETypes": ["text/csv"],
        "SupportedRealtimeInferenceInstanceTypes": ['ml.t2.medium'] 
    }
}

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model of maintenance prediction(1,0)",
    "ModelApprovalStatus" : "Approved"
}

create_model_package_input_dict.update(modelpackage_inference_specification)

print(f"\n Registrando nueva versión en el grupo {model_package_group_name}...")
create_model_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_model_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))



 Registrando nueva versión en el grupo Fleet-Model-Maintenance...


ModelPackage Version ARN : arn:aws:sagemaker:us-east-2:491934530980:model-package/Fleet-Model-Maintenance/2


In [0]:
#---------------------------------------Creacion de endpoint-------------------------------
import boto3
import time
import uuid # Para generar nombres únicos
from sagemaker import get_execution_role

# ------------- Inicialización -------------------
region = boto3.Session().region_name
role = get_execution_role() # Rol de ejecución de SageMaker
sm_client = boto3.client('sagemaker', region_name=region)

# Definición
model_package_group_name = "Fleet-Model-Maintenance"
model_version_arn = "arn:aws:sagemaker:us-east-2:491934530980:model-package/Fleet-Model-Maintenance/2"

# ----------------------------------------------------

# GENERAR NOMBRES ÚNICOS para evitar errores 'ResourceInUse'
unique_suffix = str(uuid.uuid4())[:8]
model_name = f'Fleet-Maintenance-Model-{unique_suffix}'
endpoint_config_name = f'Fleet-Maintenance-EC-{unique_suffix}'
endpoint_name = 'Fleet-Model-Maintenance-Endpoint' # Mantener el nombre si quieres que Lambda siempre apunte al mismo

# 2. CREAR EL OBJETO DEL MODELO (A partir del paquete de modelo)
print("\n1. Creando Objeto del Modelo...")
container_list = [{'ModelPackageName': model_version_arn}]

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    Containers=container_list
)
print("Model ARN : {}".format(create_model_response["ModelArn"]))

# 3. CREAR LA CONFIGURACIÓN DEL ENDPOINT
print("\n2. Creando Configuración del Endpoint...")
# IMPORTANTE: Usamos 'ml.t2.medium' (o uno de los permitidos en el registro)
instance_type = 'ml.t2.medium' 
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialVariantWeight': 1,
        'InitialInstanceCount': 1,
        'ModelName': model_name, # Apunta al objeto del modelo recién creado
        'VariantName': 'AllTraffic'
    }]
)
print("Endpoint Config ARN : {}".format(create_endpoint_config_response["EndpointConfigArn"]))

# 4. CREAR EL ENDPOINT
print("\n3. Creando Endpoint...")
# Nota: La creación de un Endpoint tarda de 5 a 10 minutos.
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)
endpoint_arn = create_endpoint_response['EndpointArn']
print(f"Endpoint ARN : {endpoint_arn}")

# 5. ESPERAR A QUE EL ENDPOINT ESTÉ ACTIVO
print("\n4. Esperando a que el Endpoint esté activo (esto tomará varios minutos)...")
while True:
    endpoint_status = sm_client.describe_endpoint(EndpointName=endpoint_name)['EndpointStatus']
    print(f"Estado: {endpoint_status}")
    if endpoint_status in ('InService', 'Failed'):
        break
    time.sleep(30)

if endpoint_status == 'InService':
    print("\n🎉 ¡DESPLIEGUE COMPLETO! El Endpoint está listo para la inferencia.")
    print(f"El nombre que usarás en Lambda es: {endpoint_name}")
else:
    print(f"\n❌ El despliegue falló. Estado: {endpoint_status}")



sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix



1. Creando Objeto del Modelo...


Model ARN : arn:aws:sagemaker:us-east-2:491934530980:model/Fleet-Maintenance-Model-f658dcd7

2. Creando Configuración del Endpoint...


Endpoint Config ARN : arn:aws:sagemaker:us-east-2:491934530980:endpoint-config/Fleet-Maintenance-EC-f658dcd7

3. Creando Endpoint...


...

Estado: Creating


Estado: Creating


Estado: Creating


Estado: Creating


Estado: InService

🎉 ¡DESPLIEGUE COMPLETO! El Endpoint está listo para la inferencia.
El nombre que usarás en Lambda es: Fleet-Model-Maintenance-Endpoint


In [0]:
import boto3

runtime = boto3.client("sagemaker-runtime")
response = runtime.invoke_endpoint(
    EndpointName="Fleet-Model-Maintenance-Endpoint",
    ContentType="text/csv",
    Body="35.2,18.1,3.19,5,87.6,180.0"
)
print(response["Body"].read().decode())


0.00852616410702467



## Shutdown cells

In [0]:
"""
Stop spark session and associated Athena Spark session
"""

from IPython import get_ipython as _get_ipython
_get_ipython().user_ns["spark"].stop()